# σ=5.0最適化実験 - 結果レポート

**実施期間**: 2026-01-08 ~ 2026-01-10  
**目的**: CMA-ESのσパラメータ調整によるL2ノルム増加とSteering効果の改善  
**対象**: 28ペルソナ + α×σ調整実験

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

## Part 1: α×σ調整実験結果

### 実験デザイン

1ペルソナ（episode-184019_A）で4条件を比較:

| 条件 | α | σ | 目的 |
|------|---|---|------|
| baseline | 2.0 | 2.0 | 現状把握 |
| high_alpha | 5.0 | 2.0 | α効果の検証 |
| high_sigma | 2.0 | 5.0 | σ効果の検証 |
| high_both | 5.0 | 5.0 | 組み合わせ効果 |

In [ ]:
# α×σ実験結果
alpha_sigma_results = {
    'Condition': ['baseline', 'high_alpha', 'high_sigma', 'high_both'],
    'α': [2.0, 5.0, 2.0, 5.0],
    'σ': [2.0, 2.0, 5.0, 5.0],
    'L2_norm': [5.237, 4.295, 10.901, 38.053],
    'Steering_win_rate': [0.40, 0.30, 0.70, 0.00],
}

df_alpha_sigma = pd.DataFrame(alpha_sigma_results)
df_alpha_sigma['L2_change_%'] = ((df_alpha_sigma['L2_norm'] / 5.237) - 1) * 100

print("="*80)
print("α×σ調整実験結果（ペルソナ: episode-184019_A）")
print("="*80)
display(df_alpha_sigma)

# 目標達成判定
print("\n🎯 目標: L2 > 5.0 AND Steering勝率 > 50%")
for _, row in df_alpha_sigma.iterrows():
    l2_ok = "✅" if row['L2_norm'] > 5.0 else "❌"
    win_ok = "✅" if row['Steering_win_rate'] > 0.5 else "❌"
    both = "✅✅ SUCCESS" if (row['L2_norm'] > 5.0 and row['Steering_win_rate'] > 0.5) else "❌ FAILED"
    print(f"{row['Condition']:<12} L2={row['L2_norm']:>7.2f} {l2_ok}  勝率={row['Steering_win_rate']:>4.0%} {win_ok}  → {both}")

In [ ]:
# 可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# L2 Norm比較
ax1 = axes[0]
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
bars = ax1.bar(df_alpha_sigma['Condition'], df_alpha_sigma['L2_norm'], color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
ax1.axhline(y=5.0, color='red', linestyle='--', linewidth=2, label='Target (L2 > 5.0)', alpha=0.7)
ax1.set_ylabel('L2 Norm', fontsize=13, fontweight='bold')
ax1.set_title('L2 Norm by Condition', fontsize=15, fontweight='bold', pad=15)
ax1.legend(fontsize=11)
ax1.grid(axis='y', alpha=0.3, linestyle='--')
ax1.set_ylim([0, 42])

for bar, val in zip(bars, df_alpha_sigma['L2_norm']):
    ax1.text(bar.get_x() + bar.get_width()/2, val + 1.5, f'{val:.2f}', 
             ha='center', va='bottom', fontweight='bold', fontsize=11)

# Steering勝率比較
ax2 = axes[1]
bars = ax2.bar(df_alpha_sigma['Condition'], df_alpha_sigma['Steering_win_rate']*100, 
               color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
ax2.axhline(y=50, color='red', linestyle='--', linewidth=2, label='Target (>50%)', alpha=0.7)
ax2.set_ylabel('Steering Win Rate (%)', fontsize=13, fontweight='bold')
ax2.set_title('Steering Win Rate by Condition', fontsize=15, fontweight='bold', pad=15)
ax2.set_ylim([0, 100])
ax2.legend(fontsize=11)
ax2.grid(axis='y', alpha=0.3, linestyle='--')

for bar, val in zip(bars, df_alpha_sigma['Steering_win_rate']*100):
    ax2.text(bar.get_x() + bar.get_width()/2, val + 3, f'{val:.0f}%', 
             ha='center', va='bottom', fontweight='bold', fontsize=11)

plt.tight_layout()
plt.savefig('../alpha_sigma_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ 保存: alpha_sigma_comparison.png")

### 🔬 重要な発見

#### ❌ 発見1: αの増加は逆効果

- **high_alpha** (α=5.0, σ=2.0): L2=4.295 (**-18%**), 勝率 30%
- **理由**: αが大きい → 小さな重みでも強い効果 → CMA-ESが小さな重みに収束

#### ✅✅ 発見2: σの増加が解決策

- **high_sigma** (α=2.0, σ=5.0): L2=10.901 (**+108%**), 勝率 **70%** ✅✅
- **理由**: σが大きい → より広い探索空間 → 大きな重みを発見できる

#### ⚠️ 発見3: 両方を上げると過剰

- **high_both**: L2=38.053 (過剰), 勝率 0% (生成崩壊)

## Part 2: σ=5.0全ペルソナ最適化結果

In [ ]:
# σ=5.0最適化結果の読み込み
results_dir = Path("../optimization_results_sigma5")

results = []
for persona_dir in sorted(results_dir.glob("episode-*")):
    summary_file = persona_dir / "summary.json"
    if summary_file.exists():
        with open(summary_file) as f:
            data = json.load(f)
            results.append({
                "persona_id": data["persona_id"],
                "best_score": data["best_score"],
                "l2_norm": data["l2_norm"],
                "iterations": data["iterations"],
                "duration_h": data["duration_seconds"] / 3600
            })

df = pd.DataFrame(results)
df['success'] = (df['best_score'] == 5.0) & (df['l2_norm'] > 5.0)

print("="*80)
print(f"σ=5.0全ペルソナ最適化結果（N={len(df)}）")
print("="*80)

# 統計
stats = {
    'L2 Norm': [df['l2_norm'].mean(), df['l2_norm'].median(), df['l2_norm'].min(), df['l2_norm'].max()],
    'Best Score': [df['best_score'].mean(), df['best_score'].median(), df['best_score'].min(), df['best_score'].max()],
}
df_stats = pd.DataFrame(stats, index=['Mean', 'Median', 'Min', 'Max'])
print("\n📊 統計サマリー:")
display(df_stats.round(2))

# 成功率
perfect = (df['best_score'] == 5.0).sum()
high_l2 = (df['l2_norm'] > 5.0).sum()
success = df['success'].sum()

print(f"\n✅ 成功率:")
print(f"  Perfect Score (5.0):    {perfect:>2}/{len(df)} ({perfect/len(df)*100:>5.1f}%)")
print(f"  L2 > 5.0:               {high_l2:>2}/{len(df)} ({high_l2/len(df)*100:>5.1f}%)")
print(f"  両方達成 (SUCCESS):     {success:>2}/{len(df)} ({success/len(df)*100:>5.1f}%)")
print(f"\n⏱ 総実行時間: {df['duration_h'].sum():.1f}時間")

In [ ]:
# 詳細テーブル
df_display = df[['persona_id', 'l2_norm', 'best_score', 'iterations', 'success']].copy()
df_display['status'] = df_display['success'].map({True: '✅', False: '⚠️'})
df_display = df_display.drop('success', axis=1)
df_display = df_display.sort_values('l2_norm', ascending=False)

print("\n📋 全ペルソナ詳細結果:")
display(df_display.style.set_properties(**{'text-align': 'left'}).format({
    'l2_norm': '{:.2f}',
    'best_score': '{:.2f}'
}))

In [ ]:
# L2 Norm分布の可視化
fig, axes = plt.subplots(2, 2, figsize=(15, 11))

# 1. L2 Norm分布（ヒストグラム）
ax1 = axes[0, 0]
ax1.hist(df['l2_norm'], bins=12, color='steelblue', alpha=0.7, edgecolor='black', linewidth=1.2)
ax1.axvline(x=5.0, color='red', linestyle='--', linewidth=2.5, label='Target (L2 > 5.0)', alpha=0.8)
ax1.axvline(x=df['l2_norm'].mean(), color='green', linestyle='--', linewidth=2.5, 
            label=f'Mean ({df["l2_norm"].mean():.2f})', alpha=0.8)
ax1.set_xlabel('L2 Norm', fontsize=12, fontweight='bold')
ax1.set_ylabel('Count', fontsize=12, fontweight='bold')
ax1.set_title('L2 Norm Distribution (N=28)', fontsize=14, fontweight='bold', pad=12)
ax1.legend(fontsize=10)
ax1.grid(axis='y', alpha=0.3)

# 2. Best Score分布
ax2 = axes[0, 1]
score_counts = df['best_score'].value_counts().sort_index()
bars = ax2.bar(score_counts.index.astype(str), score_counts.values, 
               color='coral', alpha=0.7, edgecolor='black', linewidth=1.2)
ax2.set_xlabel('Best Score', fontsize=12, fontweight='bold')
ax2.set_ylabel('Count', fontsize=12, fontweight='bold')
ax2.set_title('Best Score Distribution', fontsize=14, fontweight='bold', pad=12)
ax2.grid(axis='y', alpha=0.3)

for bar, count in zip(bars, score_counts.values):
    ax2.text(bar.get_x() + bar.get_width()/2, count + 0.5, str(count), 
             ha='center', va='bottom', fontweight='bold', fontsize=11)

# 3. L2 vs Best Score散布図
ax3 = axes[1, 0]
success_mask = df['success']
ax3.scatter(df.loc[~success_mask, 'l2_norm'], df.loc[~success_mask, 'best_score'], 
            color='lightcoral', alpha=0.7, s=120, label='Failed', edgecolors='black', linewidths=1.5)
ax3.scatter(df.loc[success_mask, 'l2_norm'], df.loc[success_mask, 'best_score'], 
            color='limegreen', alpha=0.8, s=120, label='Success', edgecolors='black', linewidths=1.5)
ax3.axvline(x=5.0, color='red', linestyle='--', alpha=0.5, linewidth=2)
ax3.axhline(y=5.0, color='red', linestyle='--', alpha=0.5, linewidth=2)
ax3.fill_between([5, 15], 5, 5.5, alpha=0.1, color='green', label='Success Zone')
ax3.set_xlabel('L2 Norm', fontsize=12, fontweight='bold')
ax3.set_ylabel('Best Score', fontsize=12, fontweight='bold')
ax3.set_title('L2 Norm vs Best Score', fontsize=14, fontweight='bold', pad=12)
ax3.legend(fontsize=10)
ax3.grid(alpha=0.3)

# 4. L2 Norm範囲別分布（パイチャート）
ax4 = axes[1, 1]
ranges = [
    ((df['l2_norm'] < 5).sum(), '< 5.0\n(Too weak)'),
    (((df['l2_norm'] >= 5) & (df['l2_norm'] < 10)).sum(), '5-10\n(Acceptable)'),
    (((df['l2_norm'] >= 10) & (df['l2_norm'] < 15)).sum(), '10-15\n(Good)'),
    ((df['l2_norm'] >= 15).sum(), '> 15\n(Strong)')
]
counts = [r[0] for r in ranges]
labels = [f"{r[1]}\n{r[0]} ({r[0]/len(df)*100:.1f}%)" for r in ranges]
colors_pie = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99']

wedges, texts, autotexts = ax4.pie(counts, labels=labels, colors=colors_pie, autopct='', 
                                     startangle=90, textprops={'fontsize': 10, 'fontweight': 'bold'})
ax4.set_title('L2 Norm Distribution by Range', fontsize=14, fontweight='bold', pad=12)

plt.tight_layout()
plt.savefig('../sigma5_results_overview.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ 保存: sigma5_results_overview.png")

## 実際の出力例: 成功ペルソナの重み

In [ ]:
# 成功例のTop 3の重みを表示
success_personas = df[df['success']].sort_values('l2_norm', ascending=False).head(3)

print("="*80)
print("✅ 成功例Top 3の最適化重み")
print("="*80)

for idx, row in success_personas.iterrows():
    persona_id = row['persona_id']
    weights_file = results_dir / persona_id / f"{persona_id}_best_weights.json"
    
    with open(weights_file) as f:
        weights = json.load(f)
    
    print(f"\n📊 {persona_id}")
    print(f"   L2 Norm: {row['l2_norm']:.2f}")
    print(f"   Best Score: {row['best_score']:.2f}")
    print(f"   Weights:")
    for trait, weight in weights.items():
        print(f"      {trait}: {weight:>8.3f}")

## 結論

### ✅ 達成

1. **σ=5.0でL2平均10.05を達成**（目標5.0の2倍）
2. **26/28ペルソナ (92.9%)でL2 > 5.0**
3. **1ペルソナ実験でSteering勝率70%を達成**（baseline 40%から改善）

### ⚠️ 課題

1. 全体成功率39.3%（両条件達成）
2. 16ペルソナが早期終了（Best Score=2.5）
3. 各ペルソナのSteering勝率は未評価

### 🎯 推奨設定

**最適パラメータ**: `α=2.0, σ=5.0, max_iterations=50, population_size=8`

### 次のステップ

1. **28ペルソナ全体でのSteering効果評価**
   - Base vs Steering比較を各ペルソナで実施
2. **早期終了ペルソナの再最適化**
   - 異なる初期値、より長いiterations
3. **Adaptive trait selection**
   - ペルソナごとに最適なtrait組み合わせを選択